In [1]:
"""
Advanced Pattern Evolution Predictor
=====================================
Integrates window-729 phase detection, autocorrelation structure,
Feigenbaum transitions, and chaos-to-order evolution tracking.
"""

import numpy as np
import pandas as pd
from scipy import stats, signal
from sklearn.ensemble import RandomForestClassifier
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

class AdvancedPatternPredictor:
    """
    Sophisticated pattern predictor that tracks evolution and phase transitions.
    """

    def __init__(self):
        # Key constants discovered
        self.PHI = 1.618033988749895
        self.INV_PHI_SQUARED = 0.381966011250105
        self.FEIGENBAUM = 4.669201609
        self.CRITICAL_NUMBER = 0.00234376636
        self.FINE_STRUCTURE = 137.035999

        # Key windows and lags
        self.PHASE_WINDOW = 729
        self.HARMONIC_WINDOW = 137
        self.KEY_LAGS = [42, 45]  # Discovered autocorrelation peaks

        # Attractors to monitor
        self.ATTRACTORS = {
            'inv_phi_squared': 0.382,
            'phi_conjugate': 0.618,
            'unity': 1.0,
            'phi': 1.618,
            'double': 2.0,
            'half': 0.5,
            'critical': 0.48  # The discovered universal density
        }

        # Phase states
        self.PHASE_CHAOS = 0
        self.PHASE_TRANSITION = 1
        self.PHASE_CONVERGING = 2
        self.PHASE_STABLE = 3

    def analyze_complete(self, data: pd.DataFrame) -> Dict:
        """
        Complete analysis integrating all discovered patterns.
        """
        results = {
            'phase_evolution': self.track_phase_evolution(data),
            'autocorr_signals': self.leverage_autocorrelation(data),
            'feigenbaum_detection': self.detect_feigenbaum_transitions(data),
            'attractor_dynamics': self.analyze_attractor_dynamics(data),
            'trading_signals': self.generate_intelligent_signals(data),
            'chaos_metrics': self.measure_chaos_level(data)
        }

        return results

    def track_phase_evolution(self, data: pd.DataFrame) -> Dict:
        """
        Use window-729 as phase detector to track system evolution.
        """
        prices = data['Close'].values

        # Calculate rolling 729-window density
        window = self.PHASE_WINDOW
        densities_729 = []
        timestamps = []

        # Use smaller step for more granular tracking
        step = max(1, window // 10)  # 10% overlap for smooth evolution

        for i in range(0, len(prices) - window, step):
            segment = prices[i:i+window]
            binary = (np.diff(segment) > 0).astype(int)
            density = np.mean(binary)
            densities_729.append(density)
            timestamps.append(i + window)

        if len(densities_729) < 3:
            return {}

        densities_729 = np.array(densities_729)

        # Calculate rate of change (velocity)
        velocity = np.diff(densities_729)

        # Calculate acceleration (second derivative)
        acceleration = np.diff(velocity) if len(velocity) > 1 else []

        # Detect phase based on dynamics
        phases = []
        for i in range(len(densities_729)):
            phase = self._classify_phase(
                densities_729[i],
                velocity[min(i, len(velocity)-1)] if len(velocity) > 0 else 0,
                acceleration[min(i, len(acceleration)-1)] if len(acceleration) > 0 else 0
            )
            phases.append(phase)

        # Find closest attractor for each point
        attractor_distances = []
        closest_attractors = []

        for density in densities_729:
            distances = {name: abs(density - value)
                        for name, value in self.ATTRACTORS.items()
                        if 0 <= value <= 1}  # Only density-compatible attractors

            closest = min(distances.items(), key=lambda x: x[1])
            closest_attractors.append(closest[0])
            attractor_distances.append(closest[1])

        # Detect convergence patterns
        is_converging = self._detect_convergence(densities_729, attractor_distances)

        # Calculate the "mellowing" metric (reduction in volatility over time)
        mellowing_score = self._calculate_mellowing(densities_729)

        return {
            'densities_729': densities_729,
            'velocity': velocity,
            'acceleration': acceleration,
            'phases': phases,
            'closest_attractors': closest_attractors,
            'attractor_distances': attractor_distances,
            'is_converging': is_converging,
            'mellowing_score': mellowing_score,
            'current_phase': phases[-1] if phases else None,
            'current_density': densities_729[-1] if len(densities_729) > 0 else None,
            'trend': 'approaching' if is_converging else 'diverging'
        }

    def leverage_autocorrelation(self, data: pd.DataFrame) -> Dict:
        """
        Use discovered autocorrelation at lags 42, 45 for prediction.
        """
        returns = data['returns'].dropna().values

        if len(returns) < max(self.KEY_LAGS) + 10:
            return {}

        signals = []

        # For each point, look at the key lag relationships
        for i in range(max(self.KEY_LAGS), len(returns)):
            # Get returns at key lags
            lag_42_return = returns[i - 42] if i >= 42 else 0
            lag_45_return = returns[i - 45] if i >= 45 else 0

            # The 3-period difference might create interference pattern
            interference = lag_42_return - lag_45_return

            # Signal based on lag pattern
            if interference > 0.01:  # Constructive interference
                signal = 1
            elif interference < -0.01:  # Destructive interference
                signal = -1
            else:
                signal = 0

            signals.append(signal)

        # Calculate the autocorrelation strength at our key lags
        from statsmodels.tsa.stattools import acf
        acf_values = acf(returns, nlags=max(self.KEY_LAGS) + 5)

        lag_42_strength = acf_values[42] if len(acf_values) > 42 else 0
        lag_45_strength = acf_values[45] if len(acf_values) > 45 else 0

        return {
            'signals': signals,
            'lag_42_strength': lag_42_strength,
            'lag_45_strength': lag_45_strength,
            'interference_pattern': lag_42_strength - lag_45_strength,
            'signal_confidence': abs(lag_42_strength) + abs(lag_45_strength)
        }

    def detect_feigenbaum_transitions(self, data: pd.DataFrame) -> Dict:
        """
        Detect period-doubling cascades and Feigenbaum transitions.
        """
        prices = data['Close'].values

        if len(prices) < 200:
            return {}

        # Look for period doubling in local extrema
        extrema_indices = signal.argrelextrema(prices, np.greater)[0]

        if len(extrema_indices) < 10:
            return {}

        # Calculate distances between extrema (periods)
        periods = np.diff(extrema_indices)

        # Look for period doubling sequences
        doubling_ratios = []
        for i in range(1, len(periods)):
            if periods[i-1] > 0:
                ratio = periods[i] / periods[i-1]
                doubling_ratios.append(ratio)

        if not doubling_ratios:
            return {}

        # Check if ratios approach Feigenbaum constant
        doubling_ratios = np.array(doubling_ratios)

        # The Feigenbaum constant appears in the limit of period-doubling
        # Look for sequences approaching 4.669...
        feigenbaum_distances = np.abs(doubling_ratios - self.FEIGENBAUM)

        # Also check for "inverted" Feigenbaum (1/4.669 ≈ 0.214)
        inverted_feigenbaum = 1 / self.FEIGENBAUM
        inverted_distances = np.abs(doubling_ratios - inverted_feigenbaum)

        # Detect if we're in a period-doubling cascade
        is_cascade = np.any(feigenbaum_distances < 0.5) or np.any(inverted_distances < 0.1)

        # Calculate the "chaos parameter" (how close to Feigenbaum)
        chaos_parameter = np.min(feigenbaum_distances) if len(feigenbaum_distances) > 0 else 1.0

        # Check for the critical 0.00234... appearing in ratios
        critical_distances = np.abs(doubling_ratios - self.CRITICAL_NUMBER)
        has_critical_echo = np.any(critical_distances < 0.001)

        return {
            'periods': periods,
            'doubling_ratios': doubling_ratios,
            'feigenbaum_distance': np.min(feigenbaum_distances) if len(feigenbaum_distances) > 0 else None,
            'inverted_distance': np.min(inverted_distances) if len(inverted_distances) > 0 else None,
            'is_cascade': is_cascade,
            'chaos_parameter': chaos_parameter,
            'has_critical_echo': has_critical_echo,
            'approaching_chaos': chaos_parameter < 0.5
        }

    def analyze_attractor_dynamics(self, data: pd.DataFrame) -> Dict:
        """
        Analyze how the system moves between attractors.
        """
        prices = data['Close'].values

        # Use multiple windows to detect multi-scale dynamics
        windows = [self.HARMONIC_WINDOW, self.PHASE_WINDOW, 89, 233]  # Include Fibonacci

        attractor_strengths = {}

        for window in windows:
            if len(prices) < window:
                continue

            # Calculate density for this window
            binary = (np.diff(prices[-window:]) > 0).astype(int)
            density = np.mean(binary)

            # Find closest attractor
            for name, value in self.ATTRACTORS.items():
                if 0 <= value <= 1:
                    distance = abs(density - value)
                    strength = np.exp(-distance * 10)  # Exponential decay of influence

                    key = f'{name}_w{window}'
                    attractor_strengths[key] = strength

        # Identify dominant attractor
        if attractor_strengths:
            dominant = max(attractor_strengths.items(), key=lambda x: x[1])
            dominant_name = dominant[0].split('_w')[0]
            dominant_strength = dominant[1]
        else:
            dominant_name = 'none'
            dominant_strength = 0

        return {
            'attractor_strengths': attractor_strengths,
            'dominant_attractor': dominant_name,
            'dominant_strength': dominant_strength,
            'multi_attractor': len([s for s in attractor_strengths.values() if s > 0.5]) > 1
        }

    def measure_chaos_level(self, data: pd.DataFrame) -> Dict:
        """
        Measure the current chaos level in the system.
        """
        returns = data['returns'].dropna().values

        if len(returns) < 100:
            return {}

        # 1. Lyapunov exponent approximation (using return divergence)
        lyapunov_approx = np.std(returns) / np.mean(np.abs(returns) + 1e-10)

        # 2. Entropy
        hist, _ = np.histogram(returns, bins=50, density=True)
        hist = hist[hist > 0]
        shannon_entropy = -np.sum(hist * np.log2(hist + 1e-10)) / len(hist)

        # 3. Fractal dimension (using box-counting approximation)
        prices = data['Close'].values
        if len(prices) > 100:
            # Simplified box-counting
            scales = [2, 4, 8, 16, 32]
            counts = []
            for scale in scales:
                scaled = prices[::scale]
                if len(scaled) > 1:
                    boxes = len(np.unique(np.round(scaled * scale)))
                    counts.append(boxes)

            if len(counts) > 1:
                # Fractal dimension from slope
                log_scales = np.log(scales[:len(counts)])
                log_counts = np.log(counts)
                fractal_dim = -np.polyfit(log_scales, log_counts, 1)[0]
            else:
                fractal_dim = 1.5
        else:
            fractal_dim = 1.5

        # Chaos score (0 = ordered, 1 = chaotic)
        chaos_score = np.tanh(lyapunov_approx * shannon_entropy * (fractal_dim - 1))

        # Determine if we're in the "mellowing" phase
        recent_volatility = np.std(returns[-20:]) if len(returns) > 20 else np.std(returns)
        overall_volatility = np.std(returns)
        is_mellowing = recent_volatility < overall_volatility * 0.8

        return {
            'lyapunov_approx': lyapunov_approx,
            'shannon_entropy': shannon_entropy,
            'fractal_dimension': fractal_dim,
            'chaos_score': chaos_score,
            'is_mellowing': is_mellowing,
            'recent_volatility': recent_volatility,
            'volatility_ratio': recent_volatility / (overall_volatility + 1e-10)
        }

    def generate_intelligent_signals(self, data: pd.DataFrame) -> Dict:
        """
        Generate trading signals using all components.
        """
        # Get all analysis components
        phase_evo = self.track_phase_evolution(data)
        autocorr = self.leverage_autocorrelation(data)
        feigenbaum = self.detect_feigenbaum_transitions(data)
        attractors = self.analyze_attractor_dynamics(data)
        chaos = self.measure_chaos_level(data)

        signals = []
        confidence_scores = []

        # Determine current market state
        if not phase_evo or not chaos:
            return {'signals': [0], 'confidence': [0]}

        # Current phase from 729-window
        current_phase = phase_evo.get('current_phase', self.PHASE_CHAOS)
        is_converging = phase_evo.get('is_converging', False)
        current_density = phase_evo.get('current_density', 0.5)

        # Chaos level
        chaos_score = chaos.get('chaos_score', 0.5)
        is_mellowing = chaos.get('is_mellowing', False)

        # Dominant attractor
        dominant_attractor = attractors.get('dominant_attractor', 'none')
        attractor_strength = attractors.get('dominant_strength', 0)

        # Feigenbaum cascade detection
        approaching_chaos = feigenbaum.get('approaching_chaos', False)

        # Generate signals based on market state
        n_signals = min(len(data), 100)  # Generate reasonable number of signals

        for i in range(n_signals):
            signal = 0
            confidence = 0

            # Trading logic based on discoveries
            if current_phase == self.PHASE_CONVERGING and is_converging:
                # System is converging to attractor - trade toward it
                if dominant_attractor == 'inv_phi_squared' and current_density > 0.382:
                    signal = -1  # Sell to converge down
                    confidence = attractor_strength
                elif dominant_attractor == 'phi_conjugate' and current_density < 0.618:
                    signal = 1  # Buy to converge up
                    confidence = attractor_strength
                elif dominant_attractor == 'unity':
                    # Mean reversion
                    if current_density > 0.5:
                        signal = -1
                    else:
                        signal = 1
                    confidence = attractor_strength * 0.8

            elif current_phase == self.PHASE_STABLE and is_mellowing:
                # System is stable and mellowing - can use autocorrelation
                if 'signals' in autocorr and i < len(autocorr['signals']):
                    signal = autocorr['signals'][i]
                    confidence = autocorr.get('signal_confidence', 0.5)

            elif approaching_chaos and not is_mellowing:
                # Approaching chaos - reduce position or stay out
                signal = 0
                confidence = 0.1

            # Modulate by chaos score
            if chaos_score > 0.7:
                # High chaos - reduce confidence
                confidence *= 0.5
                if abs(signal) > 0:
                    signal = signal * 0.5  # Reduce position size

            # Check for critical number appearance
            if phase_evo.get('attractor_distances', []):
                min_distance = min(phase_evo['attractor_distances'])
                if abs(min_distance - self.CRITICAL_NUMBER) < 0.001:
                    # Critical number detected - strong signal
                    confidence = min(confidence * 2, 1.0)

            signals.append(signal)
            confidence_scores.append(confidence)

        # Calculate expected Sharpe based on conditions
        expected_sharpe = 0
        if is_converging and is_mellowing and chaos_score < 0.3:
            expected_sharpe = 0.5  # Good conditions
        elif current_phase == self.PHASE_STABLE:
            expected_sharpe = 0.3  # Decent conditions
        elif approaching_chaos:
            expected_sharpe = -0.1  # Poor conditions

        return {
            'signals': signals,
            'confidence': confidence_scores,
            'mean_confidence': np.mean(confidence_scores),
            'expected_sharpe': expected_sharpe,
            'trading_mode': self._get_trading_mode(current_phase, chaos_score, is_mellowing),
            'recommendation': self._get_recommendation(phase_evo, chaos, attractors)
        }

    def _classify_phase(self, density: float, velocity: float, acceleration: float) -> int:
        """Classify current phase based on dynamics."""

        # Check if we're near an attractor
        min_distance = min(abs(density - v) for v in self.ATTRACTORS.values() if 0 <= v <= 1)

        if min_distance < 0.05 and abs(velocity) < 0.001:
            return self.PHASE_STABLE
        elif abs(acceleration) > 0.0001 and abs(velocity) > 0.005:
            return self.PHASE_CHAOS
        elif min_distance < 0.1 and abs(velocity) < 0.01:
            return self.PHASE_CONVERGING
        else:
            return self.PHASE_TRANSITION

    def _detect_convergence(self, densities: np.ndarray, distances: List[float]) -> bool:
        """Detect if system is converging to attractor."""
        if len(distances) < 3:
            return False

        # Check if distances are decreasing
        recent_distances = distances[-5:] if len(distances) >= 5 else distances

        # Linear regression on distances
        x = np.arange(len(recent_distances))
        slope = np.polyfit(x, recent_distances, 1)[0]

        return slope < -0.001  # Negative slope = converging

    def _calculate_mellowing(self, densities: np.ndarray) -> float:
        """Calculate how much the system is 'mellowing' (stabilizing)."""
        if len(densities) < 10:
            return 0

        # Compare early volatility to recent volatility
        early_vol = np.std(densities[:len(densities)//2])
        recent_vol = np.std(densities[len(densities)//2:])

        if early_vol > 0:
            mellowing = 1 - (recent_vol / early_vol)
            return max(0, min(1, mellowing))
        return 0

    def _get_trading_mode(self, phase: int, chaos_score: float, is_mellowing: bool) -> str:
        """Determine appropriate trading mode."""
        if phase == self.PHASE_STABLE and is_mellowing:
            return "FULL_POSITION"
        elif phase == self.PHASE_CONVERGING and chaos_score < 0.5:
            return "ACCUMULATE"
        elif phase == self.PHASE_CHAOS or chaos_score > 0.7:
            return "RISK_OFF"
        else:
            return "CAUTIOUS"

    def _get_recommendation(self, phase_evo: Dict, chaos: Dict, attractors: Dict) -> str:
        """Get human-readable recommendation."""
        if not phase_evo or not chaos:
            return "Insufficient data for recommendation"

        phase = phase_evo.get('current_phase', self.PHASE_CHAOS)
        chaos_score = chaos.get('chaos_score', 0.5)
        dominant = attractors.get('dominant_attractor', 'none')

        if phase == self.PHASE_STABLE and chaos_score < 0.3:
            return f"Strong signal: System stable near {dominant} attractor"
        elif phase == self.PHASE_CONVERGING:
            return f"Moderate signal: Converging toward {dominant}"
        elif phase == self.PHASE_CHAOS:
            return "Weak signal: System in chaos, avoid trading"
        else:
            return "Neutral: System in transition, wait for clarity"


def run_advanced_test(data: pd.DataFrame) -> Dict:
    """
    Run the advanced pattern predictor on data.
    """
    print("\n" + "="*60)
    print("ADVANCED PATTERN EVOLUTION TEST")
    print("="*60)

    predictor = AdvancedPatternPredictor()
    results = predictor.analyze_complete(data)

    # Print summary
    print("\n📊 Phase Evolution:")
    if 'phase_evolution' in results and results['phase_evolution']:
        phase_evo = results['phase_evolution']
        print(f"  Current Phase: {phase_evo.get('current_phase', 'Unknown')}")
        print(f"  Current Density: {phase_evo.get('current_density', 0):.6f}")
        print(f"  Converging: {phase_evo.get('is_converging', False)}")
        print(f"  Mellowing Score: {phase_evo.get('mellowing_score', 0):.3f}")

    print("\n🔄 Autocorrelation:")
    if 'autocorr_signals' in results and results['autocorr_signals']:
        autocorr = results['autocorr_signals']
        print(f"  Lag-42 Strength: {autocorr.get('lag_42_strength', 0):.6f}")
        print(f"  Lag-45 Strength: {autocorr.get('lag_45_strength', 0):.6f}")
        print(f"  Signal Confidence: {autocorr.get('signal_confidence', 0):.3f}")

    print("\n🌀 Feigenbaum Detection:")
    if 'feigenbaum_detection' in results and results['feigenbaum_detection']:
        feig = results['feigenbaum_detection']
        print(f"  Approaching Chaos: {feig.get('approaching_chaos', False)}")
        print(f"  Chaos Parameter: {feig.get('chaos_parameter', 0):.6f}")
        print(f"  Critical Echo: {feig.get('has_critical_echo', False)}")

    print("\n🎯 Attractor Dynamics:")
    if 'attractor_dynamics' in results and results['attractor_dynamics']:
        attract = results['attractor_dynamics']
        print(f"  Dominant: {attract.get('dominant_attractor', 'none')}")
        print(f"  Strength: {attract.get('dominant_strength', 0):.3f}")

    print("\n📈 Trading Signals:")
    if 'trading_signals' in results and results['trading_signals']:
        signals = results['trading_signals']
        print(f"  Mode: {signals.get('trading_mode', 'Unknown')}")
        print(f"  Expected Sharpe: {signals.get('expected_sharpe', 0):.3f}")
        print(f"  Mean Confidence: {signals.get('mean_confidence', 0):.3f}")
        print(f"  Recommendation: {signals.get('recommendation', 'None')}")

    print("\n💫 Chaos Metrics:")
    if 'chaos_metrics' in results and results['chaos_metrics']:
        chaos = results['chaos_metrics']
        print(f"  Chaos Score: {chaos.get('chaos_score', 0):.3f}")
        print(f"  Is Mellowing: {chaos.get('is_mellowing', False)}")
        print(f"  Fractal Dimension: {chaos.get('fractal_dimension', 0):.3f}")

    return results


# Test function to validate with your data
def validate_predictor(symbol: str = 'SPY', start: str = '2020-01-01', end: str = '2024-10-31'):
    """
    Validate the predictor with real or synthetic data.
    """
    try:
        # Try to get real data
        import yfinance as yf
        data = yf.download(symbol, start=start, end=end)
        data['returns'] = data['Close'].pct_change()
        print(f"Using real data for {symbol}")
    except:
        # Generate synthetic data for testing
        print("Using synthetic data for testing")
        dates = pd.date_range(start=start, end=end, freq='D')
        prices = 100 * (1 + np.random.randn(len(dates)).cumsum() * 0.01)
        data = pd.DataFrame({
            'Close': prices,
            'Open': prices * (1 + np.random.randn(len(dates)) * 0.01),
            'High': prices * (1 + np.abs(np.random.randn(len(dates)) * 0.02)),
            'Low': prices * (1 - np.abs(np.random.randn(len(dates)) * 0.02)),
            'Volume': np.random.pareto(2, len(dates)) * 1000000,
            'returns': np.diff(prices, prepend=prices[0]) / prices
        }, index=dates)

    # Run the advanced test
    results = run_advanced_test(data)

    # Calculate actual Sharpe if we have signals
    if 'trading_signals' in results and results['trading_signals'].get('signals'):
        signals = np.array(results['trading_signals']['signals'])
        returns = data['returns'].values[-len(signals):]

        strategy_returns = signals * returns
        actual_sharpe = np.mean(strategy_returns) / (np.std(strategy_returns) + 1e-10)

        print("\n" + "="*60)
        print(f"ACTUAL SHARPE RATIO: {actual_sharpe:.4f}")
        print(f"Expected Sharpe: {results['trading_signals'].get('expected_sharpe', 0):.4f}")
        print("="*60)

    return results

# Example usage:
# results = validate_predictor('SPY', '2020-01-01', '2024-10-31')